In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

## L2A Sentinal Analysis and PCA

### DataLoader Class
Loads data from the path provided 

In [ ]:
import os

class DataLoader:
    def __init__(self) -> None:
        return
    
    def loadFromList(paths: list(str)) -> np.ndarray:
        return np.asarray([rasterio.open(path).read(1) for path in paths])
    
    def loadFromPath(path: str) -> np.ndarray:
        return rasterio.open(path).read(1)
    
    def loadFromFolder(dirPath: str) -> np.ndarray:
        paths = os.listdir(path=dirPath)
        return np.asarray([rasterio.open(dirPath+'/'+path).read(1) for path in paths])

### CustonPCA Class
Performs all the functions like
- Perform PCA
- Reconstrutc Image
- Calculate Error

In [ ]:
class CustomPCA:
    def __init__(self, n_components: int, X: np.ndarray) -> None:
        self.n_components = n_components
        self.__X = X
        self.principal_components = None
        self.mean = None
        self.components = None

    def __standardize_matrix(self, X):
        # Standardize the data
        self.mean = np.mean(self.__X)
        X_standardized = (self.__X - np.mean(self.__X)) / np.std(self.__X)
        return X_standardized                

    def fit_transform(self) -> np.ndarray:
        X_standardized = self.__standardize_matrix(self.__X)

        concat = np.concatenate(X_standardized)
        try:
            covariance_matrix = np.cov(concat, rowvar=False)
        except:
            print("Too much memory being consumed... exiting")
            exit(-1)

        # Calculate eigenvalues and eigenvectors
        eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

        # Sort eigenvalues and corresponding eigenvectors in descending order
        indices = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[indices]
        eigenvectors = eigenvectors[:, indices]

        # Select the top n_components eigenvectors
        self.components = eigenvectors[:, :self.n_components]

        # Project data onto the selected components
        self.principal_components = np.dot(concat, self.components) 
        return self.principal_components

    def inverse_transform(self, principal_components):
        # Reconstruct data from principal components
        reconstructed_data = np.dot(principal_components, self.components.T) + self.mean
        return reconstructed_data
    
    def calculate_error(self) -> float:

        if(self.principal_components == None):
            print("Calculating principal Components...")
            self.inverse_transform(self.__X)

        try:
            X_reconstructed = self.inverse_transform(self.principal_components)
            X_reconstructed = np.transpose(np.reshape(X_reconstructed, (5490, 5490, 10)), axes=(2,0,1))

            concatenated_matrix = np.concatenate(self.__X)

            mse = np.sum((self.__standardize_matrix(self.__X) - X_reconstructed) ** 2) / concatenated_matrix.size  # Divide by the total number of elements for normalization

            print("No of comps = "+str(self.n_components))
            print("MSE = " + str(mse))

            return mse
        except:
            print("Error calculation failed! please try again...")
            exit(-2)

    
    def __del__(self):
        self.components = None
        self.mean = None
        self.n_components = None
        self.__X = None
        self.principal_components = None

### Crop Image function
Crops the image to the first black pixel, for x and for y axis 

In [ ]:
def crop_image(matrix):
    x_i = 0
    y_i = 0

    for i in range(matrix.shape[0]):
        if matrix[0,i] == 0:
            y_i = i
            break
        
    for i in range(matrix.shape[1]):
        if matrix[i,0] == 0:
            x_i = i
            break

    return (x_i, y_i)

### Plot Components Function
plot any list of images that has been sent to the function

In [ ]:
def plot_components(xBands,bandLen,cmapCol):
    plt.clf()
    fig, axes = plt.subplots(1, bandLen, figsize=(15, 5))
    for i in range(bandLen):
        axes[i].imshow(xBands[i], cmap=cmapCol)
        axes[i].set_title(f'Band {i+1}')
    